

## Diffusion-Wavelet: Interpretable Diffusion and Progressive Distillation forTime Series Generation



## Necessary packages and functions call

In [1]:
import os
import torch
import numpy as np

os.chdir("C:/Users/geminia/Desktop/Diffusion-FS")
from Engine.trainer import Engine
from Utils.metric_utils import visualization
from Datasets.create_dataloader import create_dataloader
from Utils.io_utils import load_yaml_config, instantiate_from_config, build_from_teacher
from Utils.fix_tensors import clean_keys, reshape_tensors
from Models.interpretable_diffusion.model_utils import unnormalize_to_zero_to_one

### Build dataset and settings

In [ ]:
class Args_Example:
    def __init__(self) -> None:
        self.config_path = './Configs/stocks.yaml'
        self.save_dir = './Save'
        self.gpu = 0
        os.makedirs(self.save_dir, exist_ok=True)

args =  Args_Example()
configs = load_yaml_config(args.config_path)
device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')

dl_info = create_dataloader(configs, args)
model = instantiate_from_config(configs['model']).to(device) 
teacher = Engine(config=configs, args=args, model=model, dataloader=dl_info)

### Training Original Model

In [ ]:
teacher.train()

### Distillation Steps

In [5]:
##New
teacher_keys = teacher.model.state_dict()
numsteps = teacher.model.num_timesteps
changeable = clean_keys(teacher_keys,numsteps)

reshape_tensors(teacher_keys, changeable)

config_0, model_0 = build_from_teacher(config=configs, device=device)

student = Engine(config=config_0, args=args, model=model_0, dataloader=dl_info)

student.model.load_state_dict(teacher_keys)
student.model.teacher = teacher

In [ ]:
student.distill()


## Inference and Visualizations

### Sampling

In [ ]:
dataset = dl_info['dataset']
seq_length, feature_dim = dataset.window, dataset.var_num
#ori_data = np.load(os.path.join(dataset.dir, f"stock_ground_truth_{seq_length}_train.npy"))
ori_data = np.load(os.path.join(dataset.dir, f"stock_norm_truth_24_train.npy"))  # Uncomment the line if dataset other than Sine is used.
fake_data = teacher.sample(num=len(dataset), size_every=2001, shape=[seq_length, feature_dim])
if dataset.auto_norm:
    fake_data = unnormalize_to_zero_to_one(fake_data)
    np.save(os.path.join(args.save_dir, f'ddpm_fake_stock.npy'), fake_data)

In [ ]:
dataset = dl_info['dataset']
seq_length, feature_dim = dataset.window, dataset.var_num
#ori_data = np.load(os.path.join(dataset.dir, f"stock_ground_truth_{seq_length}_train.npy"))
ori_data = np.load(os.path.join(dataset.dir, f"stock_norm_truth_24_train.npy"))  # Uncomment the line if dataset other than Sine is used.

In [ ]:
fake_data = teacher.sample(num=len(dataset), size_every=4001, shape=[seq_length, feature_dim])
if dataset.auto_norm:
    fake_data = unnormalize_to_zero_to_one(fake_data)
    np.save(os.path.join(args.save_dir, f'ddpm_fake_train_stock.npy'), fake_data)

In [ ]:
fake_data_0 = student.sample(num=len(dataset), size_every=4001, shape=[seq_length, feature_dim])
if dataset.auto_norm:
    fake_data_0 = unnormalize_to_zero_to_one(fake_data_0)
    np.save(os.path.join(args.save_dir, f'ddpm_fake_stock1.npy'), fake_data_0)

### Visualization

We visualize the original and synthetic data distributions using PCA, tSNE and Density analysis.

In [ ]:
visualization(ori_data=ori_data, generated_data=fake_data, analysis='pca', compare=ori_data.shape[0])

In [ ]:
visualization(ori_data=ori_data, generated_data=fake_data, analysis='tsne', compare=ori_data.shape[0])

In [ ]:
visualization(ori_data=ori_data, generated_data=fake_data, analysis='kernel', compare=ori_data.shape[0])